In [1]:
import configparser

from azure.ai.ml import Input, load_component, MLClient
from azure.ai.ml.entities import AmlCompute
from azure.ai.ml.dsl import pipeline
from azure.identity import DefaultAzureCredential

In [2]:
config = configparser.ConfigParser()
config.read('config.ini')
subscription_id = config.get('Azure', 'subscription_id')
resource_group = config.get('Azure', 'resource_group')
workspace_name = config.get('Azure', 'workspace')
datastore_name = config.get('Azure', 'datastore_name')

In [3]:
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace_name,
)

In [4]:
cpu_compute_target = "cpu-cluster"

try:
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        type="amlcompute",
        size="Standard_D11_V2",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=180,
        tier="Dedicated",
    )
    print(
        f"AMLCompute with name {cpu_cluster.name} will be created, with compute size {cpu_cluster.size}"
    )
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

You already have a cluster named cpu-cluster, we'll reuse it as is.


In [5]:
component_names = ['description', 'clean', 'correlation', 'split', 'logistic_regression_train', 'decission_trees_train', 'score', 'evaluate']
components = {}
for name in component_names:
    component = load_component(source=f"./components/{name}_component/{name}.yml")
    components[name] = ml_client.create_or_update(component)


In [6]:
@pipeline(
    default_compute=cpu_compute_target
)
def water_potability_logistic_regression(pipeline_input_data):
    correlation_node = components['correlation'](
        data=pipeline_input_data
    )
    clean_node = components['clean'](
        data=pipeline_input_data
    )
    split_node = components['split'](
        split_data=clean_node.outputs.clean_data_output
    )
    logistic_regression_train_node = components['logistic_regression_train'](
        train_data=split_node.outputs.train_output,
        objective='Potability'
    )
    score_node = components['score'](
        model=logistic_regression_train_node.outputs.model_output,
        test_data=split_node.outputs.test_output,
    )
    evaluate_node = components['evaluate'](
        test_data=split_node.outputs.test_output,
        predict_data=score_node.outputs.predict_output,
        objective='Potability',
        target_name='Potable',
    )
    return {
        'pairplot': correlation_node.outputs.results_output,
        'model': logistic_regression_train_node.outputs.model_output,
        'report': evaluate_node.outputs.report_output
    }

In [7]:
data_asset = ml_client.data.get(name="water-potability", version='1')
water_potability = Input(type="uri_file", path=data_asset.path)
pipeline = water_potability_logistic_regression(pipeline_input_data=water_potability)

In [8]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    experiment_name='pipeline-exp',
)
ml_client.jobs.stream(pipeline_job.name)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


RunId: bubbly_lettuce_cjb56vt8l8
Web View: https://ml.azure.com/runs/bubbly_lettuce_cjb56vt8l8?wsid=/subscriptions/f3aa5221-5b34-4091-bcec-acf7b816f5b6/resourcegroups/GrpPipeline4/workspaces/Pipeline4

Streaming logs/azureml/executionlogs.txt

[2023-11-08 19:58:37Z] Submitting 2 runs, first five are: 21cab911:9b49c2f5-92a1-456b-8217-fe15256454e0,72d862ff:6e890019-285d-4914-870b-796cf2c2739d
[2023-11-08 19:58:40Z] Completing processing run id 9b49c2f5-92a1-456b-8217-fe15256454e0.
[2023-11-08 20:07:15Z] Completing processing run id 6e890019-285d-4914-870b-796cf2c2739d.
[2023-11-08 20:07:16Z] Submitting 1 runs, first five are: 5843539d:5075315e-4cf2-4e8c-af0f-fc6776225e00
[2023-11-08 20:08:05Z] Completing processing run id 5075315e-4cf2-4e8c-af0f-fc6776225e00.
[2023-11-08 20:08:06Z] Submitting 1 runs, first five are: f3750b2a:6840f6a7-230b-48c7-8cfe-2e3f3200a08e
[2023-11-08 20:08:48Z] Completing processing run id 6840f6a7-230b-48c7-8cfe-2e3f3200a08e.
[2023-11-08 20:08:48Z] Submitting 1 ru

In [10]:
output = ml_client.jobs.download(name=pipeline_job.name, download_path='./pipeline_output', all=True)

Bad pipe message: %s [b'UA\xaa\xa8\x89\x94j\xbe\xff\\\xdc\xc1z\xdc\x9cq"\\ [\x0e\x10o((DW$#\x9e\x10\xfe\xb51\x978[*\x1e\xa4\xba[i\xf7\xa6\xad\xb2\xd9\x9a%g\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00']
Bad pipe message: %s [b'']
Bad pipe message: %s [b"\x0e\xe2\x93\xd2\xff\xb5\x96\xfe\x82\xdd%1KJ\x88U\xca\x9c\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P"]
Bad pipe message: %s [b'F\xacSU\x0e0\xcaG\x83\xd2\xbc]\x18[\x97\x1c+1\x00\x00\xa2\xc0']
Bad pipe message: %s [b'\n\x009\x008\x007\x006\x00\x88\x00\x87\x00\x86\x00\x85\xc0\x19']
Bad pipe message: %s [b'\xfe\x18\r\xaf6\x98.O\x10\x12\xc4^\xd6\x932i\xfe\t\x00\x00\xa2\xc0\x14\